# Lab 1: Three Basic HR Agents

**Learning Objectives:**
- Build Agent 1: No tools, No memory (Basic chatbot)
- Build Agent 2: With tools, No memory (Data lookup)
- Build Agent 3: With tools, With memory (Full conversational agent)

**Time:** 30 minutes

## Setup: Install Dependencies

In [ ]:
!pip install -qU langchain langchain-openai langgraph

## Setup: Configure OpenAI API Key

In [ ]:
import getpass
import os

# Set your OpenAI API key
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("🔑 Enter your OpenAI API key: ")

print("✅ API key configured!")

---
# Agent 1: WITHOUT Tools and WITHOUT Memory

**Features:**
- ❌ No tools
- ❌ No memory
- ✅ Can answer general questions using LLM knowledge

In [ ]:
from langchain.agents import create_agent

# Create Agent 1: No tools, No memory
agent_1 = create_agent(
    model="openai:gpt-4o-mini",
    tools=[],  # Empty list = No tools!
    prompt="You are an HR assistant. You can chat and answer general HR questions.",
)

print("✅ Agent 1 created: NO tools, NO memory")
print("="*60)

### Test Agent 1

In [ ]:
# Test Agent 1 with a general HR question
result = agent_1.invoke({
    "messages": [{"role": "user", "content": "What are common HR policies?"}]
})

print("Question: What are common HR policies?")
print("\nAgent 1 Response:")
print(result['messages'][-1].content)

In [ ]:
# Try a follow-up question - NO MEMORY, so won't remember context
result = agent_1.invoke({
    "messages": [{"role": "user", "content": "Can you give me 3 examples?"}]
})

print("Question: Can you give me 3 examples?")
print("\nAgent 1 Response:")
print(result['messages'][-1].content)
print("\n⚠️ Notice: Agent doesn't know what examples to give (no memory of previous question)")

---
# Define Tools (for Agent 2 and 3)

We'll create two simple tools:
1. **get_employee_info** - Look up employee details
2. **check_leave_balance** - Check leave days remaining

In [ ]:
# Tool 1: Get Employee Information
def get_employee_info(employee_id: str) -> str:
    """Get employee information by ID."""
    employees = {
        "101": "Priya Sharma - Engineering - Senior Developer",
        "102": "Rahul Verma - Engineering - Manager",
        "103": "Anjali Patel - HR - Director",
        "104": "Arjun Reddy - Sales - Team Lead",
        "105": "Sneha Gupta - Marketing - Specialist"
    }
    return employees.get(employee_id, f"Employee {employee_id} not found")

# Tool 2: Check Leave Balance
def check_leave_balance(employee_id: str) -> str:
    """Check remaining leave days for an employee by ID."""
    leave_data = {
        "101": "Priya Sharma has 12 days of leave remaining",
        "102": "Rahul Verma has 8 days of leave remaining",
        "103": "Anjali Patel has 15 days of leave remaining",
        "104": "Arjun Reddy has 10 days of leave remaining",
        "105": "Sneha Gupta has 5 days of leave remaining"
    }
    return leave_data.get(employee_id, f"Leave data for employee {employee_id} not found")

print("✅ Tools defined:")
print("   1. get_employee_info - Look up employee details")
print("   2. check_leave_balance - Check leave balance")
print("\n📋 Employee Database:")
print("   101: Priya Sharma")
print("   102: Rahul Verma")
print("   103: Anjali Patel")
print("   104: Arjun Reddy")
print("   105: Sneha Gupta")

---
# Agent 2: WITH Tools but WITHOUT Memory

**Features:**
- ✅ Has tools (can access employee data)
- ❌ No memory (each question is independent)
- ✅ Good for one-time lookups

In [ ]:
# Create Agent 2: With tools, No memory
agent_2 = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_employee_info, check_leave_balance],  # Has tools!
    prompt="You are an HR assistant. Use tools to help users find employee information and check leave balances.",
    # No checkpointer = No memory!
)

print("✅ Agent 2 created: HAS tools, NO memory")
print("="*60)

### Test Agent 2 - Tool Usage

In [ ]:
# Test: Ask about employee info
result = agent_2.invoke({
    "messages": [{"role": "user", "content": "Who is employee 101?"}]
})

print("Question: Who is employee 101?")
print("\nAgent 2 Response:")
print(result['messages'][-1].content)
print("\n✅ Agent used get_employee_info tool!")

In [ ]:
# Test: Ask about leave balance
result = agent_2.invoke({
    "messages": [{"role": "user", "content": "How many leave days does employee 102 have?"}]
})

print("Question: How many leave days does employee 102 have?")
print("\nAgent 2 Response:")
print(result['messages'][-1].content)
print("\n✅ Agent used check_leave_balance tool!")

### Test Agent 2 - No Memory Problem

In [ ]:
# Try a follow-up question - NO MEMORY
result = agent_2.invoke({
    "messages": [{"role": "user", "content": "What department is she in?"}]
})

print("Question: What department is she in?")
print("\nAgent 2 Response:")
print(result['messages'][-1].content)
print("\n⚠️ Problem: Agent doesn't know who 'she' refers to (no memory!)")

---
# Agent 3: WITH Tools AND WITH Memory

**Features:**
- ✅ Has tools (can access employee data)
- ✅ Has memory (remembers conversation)
- ✅ Can handle follow-up questions
- ✅ Best for real conversations

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

# Create memory
checkpointer = InMemorySaver()

# Create Agent 3: With tools AND with memory
agent_3 = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_employee_info, check_leave_balance],  # Has tools!
    prompt="You are an HR assistant. Use tools to help users find employee information and check leave balances.",
    checkpointer=checkpointer  # Has memory!
)

print("✅ Agent 3 created: HAS tools, HAS memory")
print("="*60)

### Test Agent 3 - With Memory

In [ ]:
# Start a conversation with thread_id
config = {"configurable": {"thread_id": "conversation_1"}}

# First question
result = agent_3.invoke(
    {"messages": [{"role": "user", "content": "Who is employee 101?"}]},
    config
)

print("Question 1: Who is employee 101?")
print("\nAgent 3 Response:")
print(result['messages'][-1].content)

In [ ]:
# Follow-up question - MEMORY WORKS!
result = agent_3.invoke(
    {"messages": [{"role": "user", "content": "What's her leave balance?"}]},
    config  # Same thread_id
)

print("Question 2: What's her leave balance?")
print("\nAgent 3 Response:")
print(result['messages'][-1].content)
print("\n✅ Success: Agent remembers we're talking about Priya Sharma (employee 101)!")

In [ ]:
# Another follow-up
result = agent_3.invoke(
    {"messages": [{"role": "user", "content": "What department is she in?"}]},
    config
)

print("Question 3: What department is she in?")
print("\nAgent 3 Response:")
print(result['messages'][-1].content)
print("\n✅ Success: Agent still remembers the context!")

### Test Agent 3 - Complex Memory

In [ ]:
# Ask about another employee
result = agent_3.invoke(
    {"messages": [{"role": "user", "content": "Now tell me about employee 102"}]},
    config
)

print("Question 4: Now tell me about employee 102")
print("\nAgent 3 Response:")
print(result['messages'][-1].content)

In [ ]:
# Ask to compare - needs memory of BOTH employees!
result = agent_3.invoke(
    {"messages": [{"role": "user", "content": "Who has more leave days, the first employee or this one?"}]},
    config
)

print("Question 5: Who has more leave days, the first employee or this one?")
print("\nAgent 3 Response:")
print(result['messages'][-1].content)
print("\n✅ Amazing: Agent remembers BOTH employee 101 (Priya) AND 102 (Rahul)!")

---
# Comparison: All Three Agents

## Summary Table

| Feature | Agent 1 | Agent 2 | Agent 3 |
|---------|---------|---------|----------|
| **Tools** | ❌ No | ✅ Yes | ✅ Yes |
| **Memory** | ❌ No | ❌ No | ✅ Yes |
| **Access employee data** | ❌ No | ✅ Yes | ✅ Yes |
| **Remember conversation** | ❌ No | ❌ No | ✅ Yes |
| **Handle follow-ups** | ❌ No | ❌ No | ✅ Yes |
| **Use case** | General chat | One-time lookups | Full HR assistant |

In [ ]:
# Print detailed comparison
print("="*80)
print("DETAILED COMPARISON")
print("="*80)

print("\n🤖 Agent 1: Basic Chatbot")
print("   ✅ Can answer general HR questions")
print("   ❌ Cannot access employee database")
print("   ❌ Cannot remember conversation")
print("   💡 Best for: General HR advice")

print("\n🤖 Agent 2: Data Lookup Agent")
print("   ✅ Can access employee database")
print("   ✅ Can check leave balances")
print("   ❌ Cannot remember conversation")
print("   💡 Best for: One-time data queries")

print("\n🤖 Agent 3: Full Conversational Agent")
print("   ✅ Can access employee database")
print("   ✅ Can check leave balances")
print("   ✅ Can remember conversation")
print("   ✅ Can handle follow-up questions")
print("   💡 Best for: Real HR assistant conversations")

print("\n="*80)

## Side-by-Side Test

In [ ]:
test_question = "Tell me about employee 103"

print("="*80)
print(f"Testing all agents with: '{test_question}'")
print("="*80)

# Agent 1
print("\n🤖 Agent 1 (No tools, No memory):")
print("-" * 60)
result1 = agent_1.invoke({
    "messages": [{"role": "user", "content": test_question}]
})
print(result1['messages'][-1].content)

# Agent 2
print("\n🤖 Agent 2 (With tools, No memory):")
print("-" * 60)
result2 = agent_2.invoke({
    "messages": [{"role": "user", "content": test_question}]
})
print(result2['messages'][-1].content)

# Agent 3
print("\n🤖 Agent 3 (With tools, With memory):")
print("-" * 60)
test_config = {"configurable": {"thread_id": "test_thread"}}
result3 = agent_3.invoke(
    {"messages": [{"role": "user", "content": test_question}]},
    test_config
)
print(result3['messages'][-1].content)

print("\n" + "="*80)

## Follow-up Test (Only Agent 3 can handle this!)

In [ ]:
followup_question = "What's their leave balance?"

print("="*80)
print(f"Follow-up question: '{followup_question}'")
print("="*80)

print("\n🤖 Agent 3 (With tools, With memory):")
print("-" * 60)
result3_followup = agent_3.invoke(
    {"messages": [{"role": "user", "content": followup_question}]},
    test_config  # Same thread!
)
print(result3_followup['messages'][-1].content)

print("\n✅ Only Agent 3 can handle this follow-up because it has MEMORY!")
print("   It remembers we were talking about employee 103 (Anjali Patel)")
print("\n" + "="*80)

---
# Try It Yourself!

Now it's your turn! Try asking your own questions to each agent.

In [ ]:
# Customize this cell with your own questions!

my_question = "Who is employee 104?"  # Change this to your question
my_followup = "What about their leave?"  # Change this to your follow-up

# Test with Agent 3 (best one)
my_config = {"configurable": {"thread_id": "my_conversation"}}

print(f"Your question: {my_question}")
result = agent_3.invoke(
    {"messages": [{"role": "user", "content": my_question}]},
    my_config
)
print(f"\nResponse: {result['messages'][-1].content}")

print(f"\n\nYour follow-up: {my_followup}")
result = agent_3.invoke(
    {"messages": [{"role": "user", "content": my_followup}]},
    my_config
)
print(f"\nResponse: {result['messages'][-1].content}")

---
# Conclusion

**What you learned:**
1. ✅ How to create a basic agent (no tools, no memory)
2. ✅ How to add tools to an agent (for data access)
3. ✅ How to add memory to an agent (for conversations)
4. ✅ The difference between each approach

**Key Takeaways:**
- **Tools** = Access to data/functions
- **Memory** = Remember conversation context
- **Best practice**: Start simple, add features as needed

**Next Steps:**
- Try adding more tools (e.g., update_leave_balance)
- Connect to a real database instead of hardcoded data
- Add more complex conversation flows
- Deploy as a web service

---
**Created with:** LangChain + OpenAI + LangGraph